In [184]:
from typing import List
from enum import Enum, IntEnum
import random

from collections import Counter

class Suits(IntEnum):
        
        Green = 0,
        Yellow = 1,
        Red = 2,
        Multicolor = 3

        def __str__(self):
            return self.name
        
class HandType(IntEnum):
    HIGH_CARD = 0
    PAIR = 1
    TWO_PAIRS = 2
    THREE_OF_A_KIND = 3
    STRAIGHT = 4
    FLUSH = 5
    FULL_HOUSE = 6
    STRAIGHT_FLUSH = 7
    GANG_OF_X = 8

 
SUITS = [Suits.Red,Suits.Yellow, Suits.Green]

class Card:

    ranks = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", 'Phoenix', 'Dragon']

    @staticmethod
    def build_from_str(card_str :str):
        suit_dict = {"G": Suits.Green, "Y": Suits.Yellow, "R": Suits.Red, "M": Suits.Multicolor}
        rank, suit = card_str.split("-")
        suit = suit_dict[suit]
        return Card(rank, suit)
    
    def __init__(self, rank: str, suit: Suits) -> None:

        if rank not in Card.ranks:
            raise ValueError(f"No such rank [{rank}]")
        if suit == Suits.Multicolor and rank != "1":
            raise ValueError(f"Card [{rank}] cannot be multicolor")
        if rank == 'Phoenix' and suit not in (Suits.Green, Suits.Yellow):
            raise ValueError(f"Phoenix can only be yellow or green")
        
        self.rank = rank
        self.suit = suit
        self.is_poulet = self.rank in ("Phoenix", 'Dragon')
    
    def get_rank_value(self) -> int:
        return Card.ranks.index(self.rank)
    
    def __hash__(self):
        return hash(str(self))

    def __eq__(self, value: object) -> bool:
        return (self.rank == value.rank) and (self.suit == value.suit)
    
    def __lt__(self, other:object) -> bool:
        rank_a_idx = self.get_rank_value()
        rank_b_idx = other.get_rank_value()
        return (rank_a_idx < rank_b_idx) or ((rank_a_idx == rank_b_idx) and (self.suit < other.suit))
    
    def __le__(self, other) -> bool:
        return self.__lt__(other) or self.__eq__(other)
        
    
    def __str__(self) -> str:
        return f"{self.rank}-{str(self.suit)[0]}"
    
    def __repr__(self) -> str:
        return f"{self.rank}-{str(self.suit)[0]}"
    

class Deck:


    def __init__(self) -> None:
        self.suits = [Suits.Red,Suits.Yellow, Suits.Green]
        self.ranks = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
        self.special_cards = [('1', Suits.Multicolor), ('Phoenix', Suits.Green),
                              ('Phoenix', Suits.Yellow), ('Dragon', Suits.Red)]
        
        self.deck = self.__create_deck()

    def __create_deck(self) -> List[Card]:
        """Create a complete deck"""
        card_list = [Card(rank=rank, suit=suit) for suit in self.suits for rank in self.ranks]
        card_list += [Card(rank=rank, suit=suit) for suit in self.suits for rank in self.ranks]
        card_list += [Card(rank=rank, suit=suit) for rank,suit in self.special_cards]

        return card_list
    
    def shuffle(self):        
        random.shuffle(self.deck)

    def nb_cards(self):
        return len(self.deck)
    
    def deal_card(self, nb_card:int) -> List[Card]:
        if nb_card > self.nb_cards():
            raise ValueError("Not enought cards remaining")
        return [self.deck.pop() for _ in range(nb_card)]
    
class Hand:
    # rank_order = '12345678910PhoenixDragon'
    rank_order = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", 'Phoenix', 'Dragon']

    @staticmethod
    def build_from_str(card_str_list: List[str]):
        result_card_list : List[Card] = []
        for card_str in card_str_list:
            result_card_list.append(Card.build_from_str(card_str))

        return Hand(result_card_list)
    
    # @staticmethod
    # def sort_cards(card_list: List[Card], sort_method):


    
    def __init__(self, cards: List[Card]):
        
        self.cards = tuple(sorted(cards))
        self.has_poulet = any(card.is_poulet for card in self.cards)
        self.ranks = [card.rank for card in self.cards]
        self.suits = [card.suit for card in self.cards]

        self.hand_type = self.calculate_hand_type()

        if self.hand_type is None:
            raise ValueError("Not a valid hand")
        
    def contains(self, card: Card):
        return card in self.cards

    def get_hand_size(self) -> int:
        return len(self.cards)
    
    def get_card_list(self) -> List[Card]:
        return list(self.cards)
    
    def get_str_card_list(self):
        return [str(card) for card in self.cards]

    def check_valid_five_combination(self):
        
        if self.has_poulet:
            return False
        else:
            return len(self.cards) == 5
        
    def is_flush(self):
        """Check if all cards have the same suit."""
        suits = self.suits.copy()
        # multicolor works as any color
        if Suits.Multicolor in suits:
            suits.remove(Suits.Multicolor)

        return (len(set(suits)) == 1) and self.check_valid_five_combination()

    def is_straight(self):
        """Check if the cards form a sequence."""
        if not self.check_valid_five_combination():
            return False
        int_ranks = [int(rank) for rank in self.ranks]
        return (int_ranks == list(range(int_ranks[0], int_ranks[0] + 5)))

    def is_gank_of_x(self):
        """Check if there are four cards of the same rank."""
        return (len(self.cards) >= 4 and len(set(self.ranks)) == 1)
    

    def is_full_house(self):
        """Check for a full house (three of a kind and a pair)."""
        if not len(self.cards) == 5:
            return False
        
        rank_counts = Counter(self.ranks)
        return sorted(rank_counts.values()) == [2, 3]

    def is_three_of_a_kind(self):
        """Check if there are three cards of the same rank."""
        # if self.get_hand_size() != 3:
        #     return False
        
        rank_counts = Counter(self.ranks)
        return list(rank_counts.values()) == [3]

    def is_two_pair(self):
        """Check if there are two pairs."""

        rank_counts = Counter(self.ranks)
        return list(rank_counts.values()) == [2,2]

    def is_one_pair(self):
        """Check if there is one pair."""
        rank_counts = Counter(self.ranks)
        return list(rank_counts.values()) == [2]
    
    def is_high_card(self):
        return len(self.cards) == 1

    def calculate_hand_type(self) -> HandType:
        if self.is_gank_of_x():
            return HandType.GANG_OF_X
        
        if self.is_flush() and self.is_straight():
            return HandType.STRAIGHT_FLUSH
        
        if self.is_full_house():
            return HandType.FULL_HOUSE
        
        if self.is_flush():
            return HandType.FLUSH

        if self.is_straight():
            return HandType.STRAIGHT

        if self.is_three_of_a_kind():
            return HandType.THREE_OF_A_KIND
    
        if self.is_two_pair():
            return HandType.TWO_PAIRS
    
        if self.is_one_pair():
            return HandType.PAIR

        if self.is_high_card():
            return HandType.HIGH_CARD
        
        return None
    
    def valid_to_play(self, previous_hand):
        if previous_hand is None:
            return True
    
        if not ((self.hand_type == HandType.GANG_OF_X) or (self.get_hand_size() == self.previous_hand.get_hand_size())):
            return False
        
        return previous_hand.__lt__(self)
    
    def __eq__(self, other):
        """Equality comparison between two hands."""
        return self.cards == other.cards

    def __lt__(self, other: object):
        """Less than comparison between two hands."""
        if type(other) != Hand:
            raise ValueError(f"Cannot compare Hand and [{type(other)}]")
     
        if (self.get_hand_size() != other.get_hand_size()) and not (HandType.GANG_OF_X in [self.hand_type, other.hand_type]):
            raise ValueError(f"Cannot compare hands of different lenghts")

        
        if self.hand_type < other.hand_type:
            return True
        
        elif self.hand_type == other.hand_type:

            if self.hand_type == HandType.GANG_OF_X:
                if self.get_hand_size() != other.get_hand_size():
                    # Gang of 5 < Gang of 6 
                    return self.get_hand_size() < other.get_hand_size()
                else:
                    # Check rank of cards
                    return self.cards[0] < other.cards[0]
                
            # Hands have same number of cards
            for self_card, other_card in zip(*map(reversed, (self.cards, other.cards))):
                if self_card != other_card:
                    return (self_card < other_card)
            
            # All cards are equal
        return False
    
    def __le__(self, other):
        return (self.__lt__(other) or self.__eq__(other))
    
    def __hash__(self):
        return hash(self.cards)

    def __str__(self):
        return str(self.cards)
    
    def __repr__(self):
        return self.__str__()

In [185]:
import copy

deck = Deck()
deck.shuffle()
card_list = deck.deal_card(16)
print(card_list)


[4-G, 3-Y, 4-Y, 9-Y, Dragon-R, 2-R, 5-G, 6-G, 8-Y, 2-Y, 2-R, 6-Y, 5-R, 8-R, 8-Y, 4-R]


In [186]:
card_list = [
        Card("2", Suits.Green),
        
        Card("1", Suits.Multicolor),
        Card("1", Suits.Green),
        Card("2", Suits.Green),
        Card("1", Suits.Red),
        Card("1", Suits.Red),
        Card("1", Suits.Yellow),
        Card("3", Suits.Green),
        Card("4", Suits.Green),
        Card("2", Suits.Red),
        Card("2", Suits.Red),
        Card("2", Suits.Yellow),
        Card("5", Suits.Green),
        Card("6", Suits.Green),
        Card("7", Suits.Green),
        Card("8", Suits.Green),
        Card("9", Suits.Green)]

In [ ]:
from itertools import combinations
from collections.abc import Iterable
from copy import deepcopy

def get_pairs(card_list: Iterable[Card]) -> set[Hand]:
    pairs = set()
    value_counter = Counter(card.get_rank_value() for card in card_list)
    for value, count in value_counter.items():
        if count >= 2:
            relevent_cards = [card for card in card_list if card.get_rank_value() == value]
            pairs.update(Hand(comb) for comb in combinations(relevent_cards, 2))
    return pairs

def get_three_of_a_kinds(card_list : Iterable[Card]) -> set[Hand]:
    triplets = set()
    value_counter = Counter(card.get_rank_value() for card in card_list)
    for value, count in value_counter.items():
        if count >= 3:
            relevent_cards = [card for card in card_list if card.get_rank_value() == value]
            triplets.update(Hand(comb) for comb in combinations(relevent_cards, 3))
    return triplets

def get_gang_of_x(card_list: Iterable[Card]) -> set[Hand]:
    gang_of_x_list = set()
    value_counter = Counter(card.get_rank_value() for card in card_list)
    for value, count in value_counter.items():
        if count >=4:
            relevent_cards = [card for card in card_list if card.get_rank_value() == value]
            for x in range(4, count + 1):
                gang_of_x_list.update(Hand(comb) for comb in combinations(relevent_cards, x))

    return gang_of_x_list

def get_two_pairs(card_list : Iterable[Card]) -> set[Hand]:
    pairs = get_pairs(card_list)

    two_pairs = set()
    pairs = [pair.get_card_list() for pair in pairs]
    for first_pair_idx in range(len(pairs) -1):
        for second_pair_idx in range(first_pair_idx, len(pairs)):
            first_pair = pairs[first_pair_idx]
            second_pair = pairs[second_pair_idx]
            if first_pair[0].get_rank_value() != second_pair[0].get_rank_value():
                two_pairs.update([Hand(first_pair + second_pair)])
    return two_pairs

def get_full_houses(card_list: Iterable[Card]) -> set[Hand]:
    # value_counter = Counter(card.get_rank_value() for card in card_list)
    pairs = get_pairs(card_list)
    three_of_a_kinds = get_three_of_a_kinds(card_list)    

    full_house_list = set()
    for pair in pairs:
        pair = pair.get_card_list()
        for three_of_a_kind in three_of_a_kinds:
            three_of_a_kind = three_of_a_kind.get_card_list()
            if pair[0].get_rank_value() != three_of_a_kind[0].get_rank_value():
                full_house_list.update([Hand(pair + three_of_a_kind)])

    return full_house_list

def get_flushes(card_list: Iterable[Card]) -> set[Hand]:
    flushes = set()
    card_list = [card for card in card_list if not card.is_poulet]
    suit_counter = Counter(card.suit for card in card_list)
    for suit, count in suit_counter.items():
        if (count + suit_counter[Suits.Multicolor] >=5):
            relevent_cards = (card for card in card_list if card.suit in (suit, Suits.Multicolor))
            flushes.update(Hand(comb) for comb in combinations(relevent_cards, 5))
    return flushes


def get_straights(card_list: Iterable[Card]) -> set[Hand]:

    card_list = [card for card in card_list if not card.is_poulet]
    value_counter = Counter(card.get_rank_value() for card in card_list)
    values = sorted(value_counter.keys())

    straight_set = set()
    
    for key_idx in range(len(values) -4):
        start_value = values[key_idx]
        end_value = values[key_idx + 4]

        if  start_value == end_value - 4:
            # There is a straight
            new_straights = [[]]
                
            for rank_value in range(start_value, end_value + 1):
                
                rank_cards = list(set(card for card in card_list if card.get_rank_value() == rank_value))

                nb_cards_to_add = len(rank_cards)
                # Add nb_new_cards - 1 list to to the list of straights to cover all possibilities
                new_straights = [straight.copy() for straight in new_straights for _ in range(nb_cards_to_add)]

                for straight_idx in range(len(new_straights)):

                    card_to_add = rank_cards[straight_idx % (nb_cards_to_add) if (nb_cards_to_add >= 1) else 0]

                    new_straights[straight_idx].append(card_to_add)


            straight_set.update(Hand(straight) for straight in new_straights)
    
    return straight_set


def get_possible_combinations(card_list : Iterable[Card], last_hand:Hand) -> List[Hand]:
    possible_combinations: set[Hand] = set() # Set of Hands objects
    
    # SINGLE CARD
    if last_hand is None or last_hand.hand_type == HandType.HIGH_CARD:
        possible_combinations.update([Hand([card]) for card in card_list])
    
    if last_hand is None or last_hand.hand_type == HandType.PAIR:
        possible_combinations.update(get_pairs(card_list))

    if last_hand is None or last_hand.hand_type == HandType.THREE_OF_A_KIND:
        possible_combinations.update(get_three_of_a_kinds(card_list))
    
    # Gang Of Fours:
    possible_combinations.update(get_gang_of_x(card_list))
    
    # Two pairs
    if last_hand is None or (last_hand.hand_type == HandType.TWO_PAIRS):
        possible_combinations.update(get_two_pairs(card_list))

    if last_hand is None or (last_hand.hand_type in (HandType.FLUSH, HandType.STRAIGHT, HandType.FULL_HOUSE)):
        possible_combinations.update(get_full_houses(card_list))

    # Flushes (includes straight flushes)
    if last_hand is None or (last_hand.hand_type in (HandType.FLUSH, HandType.STRAIGHT, HandType.FULL_HOUSE, HandType.STRAIGHT_FLUSH)):
        possible_combinations.update(get_flushes(card_list))

    # Straight (straight flushes already included)
    if last_hand is None or (last_hand.hand_type in HandType.STRAIGHT):
        straight_set = get_straights(card_list)
        possible_combinations.update(straight_set)
        
    
    return [hand for hand in possible_combinations if hand.valid_to_play(last_hand)]
    

In [204]:
playable_combs = get_possible_combinations(hand, None)
sorted(playable_combs, key=lambda x: (x.hand_type, x))

[(1-G,),
 (1-Y,),
 (1-R,),
 (1-M,),
 (2-G,),
 (2-Y,),
 (2-R,),
 (3-G,),
 (4-G,),
 (5-G,),
 (6-G,),
 (7-G,),
 (8-G,),
 (9-G,),
 (1-G, 1-Y),
 (1-G, 1-R),
 (1-Y, 1-R),
 (1-R, 1-R),
 (1-G, 1-M),
 (1-Y, 1-M),
 (1-R, 1-M),
 (2-G, 2-G),
 (2-G, 2-Y),
 (2-G, 2-R),
 (2-Y, 2-R),
 (2-R, 2-R),
 (1-G, 1-Y, 2-G, 2-G),
 (1-G, 1-R, 2-G, 2-G),
 (1-Y, 1-R, 2-G, 2-G),
 (1-R, 1-R, 2-G, 2-G),
 (1-G, 1-M, 2-G, 2-G),
 (1-Y, 1-M, 2-G, 2-G),
 (1-R, 1-M, 2-G, 2-G),
 (1-G, 1-Y, 2-G, 2-Y),
 (1-G, 1-R, 2-G, 2-Y),
 (1-Y, 1-R, 2-G, 2-Y),
 (1-R, 1-R, 2-G, 2-Y),
 (1-G, 1-M, 2-G, 2-Y),
 (1-Y, 1-M, 2-G, 2-Y),
 (1-R, 1-M, 2-G, 2-Y),
 (1-G, 1-Y, 2-G, 2-R),
 (1-G, 1-R, 2-G, 2-R),
 (1-Y, 1-R, 2-G, 2-R),
 (1-R, 1-R, 2-G, 2-R),
 (1-G, 1-M, 2-G, 2-R),
 (1-Y, 1-M, 2-G, 2-R),
 (1-R, 1-M, 2-G, 2-R),
 (1-G, 1-Y, 2-Y, 2-R),
 (1-G, 1-R, 2-Y, 2-R),
 (1-Y, 1-R, 2-Y, 2-R),
 (1-R, 1-R, 2-Y, 2-R),
 (1-G, 1-M, 2-Y, 2-R),
 (1-Y, 1-M, 2-Y, 2-R),
 (1-R, 1-M, 2-Y, 2-R),
 (1-G, 1-Y, 2-R, 2-R),
 (1-G, 1-R, 2-R, 2-R),
 (1-Y, 1-R, 2-R, 2-R),
 (1-R

In [200]:
combs = [(hand.hand_type, hand.get_card_list())for hand in get_possible_combinations(hand, None)]

In [202]:
sorted(combs, key=lambda x: (x[0], x[1]))

[(<HandType.HIGH_CARD: 0>, [1-G]),
 (<HandType.HIGH_CARD: 0>, [1-Y]),
 (<HandType.HIGH_CARD: 0>, [1-R]),
 (<HandType.HIGH_CARD: 0>, [1-M]),
 (<HandType.HIGH_CARD: 0>, [2-G]),
 (<HandType.HIGH_CARD: 0>, [2-Y]),
 (<HandType.HIGH_CARD: 0>, [2-R]),
 (<HandType.HIGH_CARD: 0>, [3-G]),
 (<HandType.HIGH_CARD: 0>, [4-G]),
 (<HandType.HIGH_CARD: 0>, [5-G]),
 (<HandType.HIGH_CARD: 0>, [6-G]),
 (<HandType.HIGH_CARD: 0>, [7-G]),
 (<HandType.HIGH_CARD: 0>, [8-G]),
 (<HandType.HIGH_CARD: 0>, [9-G]),
 (<HandType.PAIR: 1>, [1-G, 1-Y]),
 (<HandType.PAIR: 1>, [1-G, 1-R]),
 (<HandType.PAIR: 1>, [1-G, 1-M]),
 (<HandType.PAIR: 1>, [1-Y, 1-R]),
 (<HandType.PAIR: 1>, [1-Y, 1-M]),
 (<HandType.PAIR: 1>, [1-R, 1-R]),
 (<HandType.PAIR: 1>, [1-R, 1-M]),
 (<HandType.PAIR: 1>, [2-G, 2-G]),
 (<HandType.PAIR: 1>, [2-G, 2-Y]),
 (<HandType.PAIR: 1>, [2-G, 2-R]),
 (<HandType.PAIR: 1>, [2-Y, 2-R]),
 (<HandType.PAIR: 1>, [2-R, 2-R]),
 (<HandType.TWO_PAIRS: 2>, [1-G, 1-Y, 2-G, 2-G]),
 (<HandType.TWO_PAIRS: 2>, [1-G, 1-Y, 2-

In [168]:
test = Hand([Card("1", Suits.Yellow),Card("2", Suits.Red),Card("3", Suits.Green), Card("4", Suits.Green), Card("5", Suits.Green)])
test.hand_type

<HandType.THREE_OF_A_KIND: 3>

In [ ]:
get_possible_combinations(card_list, None)

{(1-G,),
 (1-Y,),
 (1-R,),
 (1-M,),
 (2-G,),
 (2-Y,),
 (2-R,),
 (3-G,),
 (4-G,),
 (5-G,),
 (6-G,),
 (7-G,),
 (8-G,),
 (9-G,)}

In [ ]:
possible_combinations = []
specials = [card for card in card_list if card.is_poulet]
hand_numbers = [card for card in card_list if not card.is_poulet]

value_counter = Counter(card.get_rank_value() for card in hand_numbers if not card.is_poulet)
suit_counter = Counter(card.suit for card in hand_numbers if not card.is_poulet)
special_counter = Counter(special.rank for special in specials)
# Single cards
possible_combinations.extend([("Single", [card]) for card in card_list])



In [60]:
from itertools import combinations

In [126]:
from typing import Tuple
from collections.abc import Iterable


def get_pairs(hand: Iterable[List]) -> set[Hand]:
    pairs = set()
    value_counter = Counter(card.get_rank_value() for card in hand)
    for value, count in value_counter.items():
        if count >= 2:
            relevent_cards = [card for card in hand if card.get_rank_value() == value]
            pairs.update(Hand(comb) for comb in combinations(relevent_cards, 2))
    return pairs

def get_three_of_a_kinds(hand : Iterable[Card]) -> set[Hand]:
    triplets = set()
    value_counter = Counter(card.get_rank_value() for card in hand)
    for value, count in value_counter.items():
        if count >= 3:
            relevent_cards = [card for card in hand if card.get_rank_value() == value]
            triplets.update(Hand(comb) for comb in combinations(relevent_cards, 3))
    return triplets

def get_gang_of_x(hand: Iterable[Card]) -> set[Hand]:
    gang_of_x_list = set()
    value_counter = Counter(card.get_rank_value() for card in hand)
    for value, count in value_counter.items():
        if count >=4:
            relevent_cards = [card for card in hand if card.get_rank_value() == value]
            for x in range(4, count + 1):
                gang_of_x_list.update(Hand(comb) for comb in combinations(relevent_cards, x))

In [ ]:
get_pairs(card_list)
three = get_three_of_a_kinds(card_list)

In [125]:
three

{(1-G, 1-Y, 1-R),
 (1-G, 1-R, 1-R),
 (1-Y, 1-R, 1-R),
 (1-G, 1-Y, 1-M),
 (1-G, 1-R, 1-M),
 (1-Y, 1-R, 1-M),
 (1-R, 1-R, 1-M),
 (2-G, 2-G, 2-Y),
 (2-G, 2-G, 2-R),
 (2-G, 2-Y, 2-R),
 (2-G, 2-R, 2-R),
 (2-Y, 2-R, 2-R)}

In [63]:
tuple_1 = (Card('4',Suits.Red), Card('4', Suits.Yellow), Card('4', Suits.Green))
tuple_2 = (Card('4',Suits.Yellow), Card('4', Suits.Red), Card('4', Suits.Green))


In [64]:
t = set()
t.update(set(list(tuple_1) + list(tuple_2)))
print(t)

{4-R, 4-G, 4-Y}


In [65]:
set((tuple_1, tuple_2))

{(4-Y, 4-R, 4-G), (4-R, 4-Y, 4-G)}

In [66]:
list(three)[2] == list(three)[0]

False

In [ ]:
from itertools import combinations
from copy import deepcopy
possible_combinations = []
last_hand = None

pairs = set()
gang_of_xs = set()
three_of_a_kinds = set()

for value, count in value_counter.items():
    
    # pairCards = [card for card in hand if card.get_rank_value() == value]
    rank_cards = [card for card in card_list if card.get_rank_value() == value]
    if count >= 2 and (last_hand is None or (HandType.PAIR == last_hand.hand_type)):
        
        pairs.update(combinations(rank_cards, 2))

        

    if count >= 3 and (last_hand is None or (HandType.THREE_OF_A_KIND == last_hand.hand_type)):
        three_of_a_kinds.update(combinations(rank_cards, 3))
        

    # Can always play GoF
    if count >=4:
        for x in range(4,count + 1):
            gang_of_xs.update(combinations(rank_cards, x))
            


possible_combinations.append(("Pair", pairs))
possible_combinations.append(("ThreeOfAKind", three_of_a_kinds))
possible_combinations.append(("GangOfX",gang_of_xs))

# Two pairs
if last_hand is None or (last_hand.hand_type == HandType.TWO_PAIRS):
    
    # pairs = []
    # for value, count in value_counter.items():
    #     relevent_cards = [card for card in hand if card.get_rank_value() == value]
    #     pairs.append(set(combinations(relevent_cards, 2)))
    two_pairs = set()
    pairs = list(pairs)
    for first_pair_idx in range(len(pairs) -1):
        for second_pair_idx in range(first_pair_idx, len(pairs)):
            first_pair = pairs[first_pair_idx]
            second_pair = pairs[second_pair_idx]
            if first_pair[0].get_rank_value() != second_pair[0].get_rank_value():
                two_pairs.update([first_pair + second_pair])
    
    possible_combinations.append(("TWO_PAIRS", two_pairs))

for three_value, three_count in value_counter.items():
pairs = get_pairs(card_list)
three_of_a_kinds = get_three_of_a_kinds(card_list)

full_house_list = []
for pair in pairs:
    for three_of_a_kind in three_of_a_kinds:
        if pair[0].get_rank_value() != three_of_a_kind[0].get_rank_value():
            full_house_list.append(pair + three_of_a_kind)

possible_combinations.append(("Full House", full_house_list))


flushes = set()
for suit, count in suit_counter.items():
    if (count + suit_counter[Suits.Multicolor] >=5):
        rank_cards = (card for card in hand_numbers if card.suit in (suit, Suits.Multicolor))
        flushes.update(tuple(sorted(comb)) for comb in combinations(rank_cards, 5))

if len(flushes) >= 1:   
    possible_combinations.append(("Flush", flushes))

# Straight
values = sorted(value_counter.keys())
print("sorted_values",values)
straight_set = set()
for key_idx in range(len(values) -4):
    start_value = values[key_idx]
    end_value = values[key_idx + 4]
    # There is a straight
    if  start_value == end_value - 4:
        new_straights = [[]]
            
        for rank_idx, rank_value in enumerate(range(start_value, end_value + 1)):
            rank_cards = list(set(card for card in hand_numbers if card.get_rank_value() == rank_value))
            nb_cards_to_add = len(rank_cards)

            new_straights = [straight.copy() for straight in new_straights for _ in range(nb_cards_to_add)]

            for straight_idx in range(len(new_straights)):

                card_to_add = rank_cards[straight_idx % (nb_cards_to_add - 1) if (nb_cards_to_add > 1) else 0]

                new_straights[straight_idx].append(card_to_add)

        straight_set.update([tuple(straight) for straight in new_straights])

if len(straight_set) >= 1:
    possible_combinations.append(("straight", straight_set))

# Straight flush:
straight_flush_list = set()
for straight in straight_set:
    suits = [card.suit for card in straight]
    # multicolor works as any color
    if Suits.Multicolor in suits:
        suits.remove(Suits.Multicolor)

    if len(set(suits)) == 1:
        straight_flush_list.update([deepcopy(straight)])

if len(straight_flush_list) >= 1:
    possible_combinations.append(("StraightFlush", straight_flush_list))

    

sorted_values [0, 1, 2, 3, 4, 5, 6, 7, 8]


In [75]:
possible_combinations[1:]

[('straight',
  {(1-G, 2-G, 3-G, 4-G, 5-G),
   (1-G, 2-R, 3-G, 4-G, 5-G),
   (1-Y, 2-G, 3-G, 4-G, 5-G),
   (1-Y, 2-R, 3-G, 4-G, 5-G),
   (1-R, 2-G, 3-G, 4-G, 5-G),
   (1-R, 2-R, 3-G, 4-G, 5-G),
   (2-G, 3-G, 4-G, 5-G, 6-G),
   (2-R, 3-G, 4-G, 5-G, 6-G),
   (3-G, 4-G, 5-G, 6-G, 7-G),
   (4-G, 5-G, 6-G, 7-G, 8-G),
   (5-G, 6-G, 7-G, 8-G, 9-G)}),
 ('StraightFlush',
  {(1-G, 2-G, 3-G, 4-G, 5-G),
   (2-G, 3-G, 4-G, 5-G, 6-G),
   (3-G, 4-G, 5-G, 6-G, 7-G),
   (4-G, 5-G, 6-G, 7-G, 8-G),
   (5-G, 6-G, 7-G, 8-G, 9-G)})]

In [ ]:
card_list = [
        Card("2", Suits.Green),
        
        Card("1", Suits.Multicolor),
        Card("1", Suits.Green),
        Card("1", Suits.Green),

        Card("2", Suits.Green),
        Card("2", Suits.Green),


        Card("2", Suits.Green)]

possible_combinations = []
specials = [card for card in card_list if card.is_poulet]
hand_numbers = [card for card in card_list if not card.is_poulet]

value_counter = Counter(card.get_rank_value() for card in hand_numbers if not card.is_poulet)
suit_counter = Counter(card.suit for card in hand_numbers if not card.is_poulet)
special_counter = Counter(special.rank for special in specials)
# Single cards




In [ ]:
possible_combinations = []
flushes = set()
for suit, count in suit_counter.items():
    if (count + suit_counter[Suits.Multicolor] >=5):
        print("hello")
        rank_cards = (card for card in hand_numbers if card.suit in (suit, Suits.Multicolor))
        # print(list(relevent_cards))
        flushes.update(set(combinations(rank_cards, 5)))

if len(flushes) > 1:   
    possible_combinations.append(("Flush", flushes))

hello


In [ ]:
rank_cards = [card for card in hand_numbers if card.suit in (Suits.Green, Suits.Multicolor)]

In [ ]:
[(1,2,3), (2,1,3), (2,3,4)]

False

In [428]:
possible_combinations[0][1]

{(1-G, 1-G, 2-G, 2-G, 2-G),
 (1-M, 1-G, 1-G, 2-G, 2-G),
 (1-M, 1-G, 2-G, 2-G, 2-G),
 (2-G, 1-G, 1-G, 2-G, 2-G),
 (2-G, 1-G, 2-G, 2-G, 2-G),
 (2-G, 1-M, 1-G, 1-G, 2-G),
 (2-G, 1-M, 1-G, 2-G, 2-G),
 (2-G, 1-M, 2-G, 2-G, 2-G)}

In [430]:
set(tuple(sorted(flush)) for flush in possible_combinations[0][1])

{(1-G, 1-G, 1-M, 2-G, 2-G),
 (1-G, 1-G, 2-G, 2-G, 2-G),
 (1-G, 1-M, 2-G, 2-G, 2-G),
 (1-G, 2-G, 2-G, 2-G, 2-G),
 (1-M, 2-G, 2-G, 2-G, 2-G)}

In [413]:
l = [(2,2),(2,3),(1,4),(2,2), (4,1)]
list(set(l))

[(2, 3), (4, 1), (1, 4), (2, 2)]

In [417]:
set(frozenset(sub) for sub in l)

{frozenset({2}), frozenset({1, 4}), frozenset({2, 3})}

In [425]:
set(tuple(sorted(sub)) for sub in l)

{(1, 4), (2, 2), (2, 3)}

In [ ]:
frozenset(combinations(rank_cards, 5))

frozenset({(1-G, 1-G, 2-G, 2-G, 2-G),
           (1-M, 1-G, 1-G, 2-G, 2-G),
           (1-M, 1-G, 2-G, 2-G, 2-G),
           (2-G, 1-G, 1-G, 2-G, 2-G),
           (2-G, 1-G, 2-G, 2-G, 2-G),
           (2-G, 1-M, 1-G, 1-G, 2-G),
           (2-G, 1-M, 1-G, 2-G, 2-G),
           (2-G, 1-M, 2-G, 2-G, 2-G)})

In [ ]:
set(combinations(rank_cards, 5))

set()

In [407]:
possible_combinations

[('Flush',
  {(1-G, 1-G, 2-G, 2-G, 2-G),
   (1-M, 1-G, 1-G, 2-G, 2-G),
   (1-M, 1-G, 2-G, 2-G, 2-G),
   (2-G, 1-G, 1-G, 2-G, 2-G),
   (2-G, 1-G, 2-G, 2-G, 2-G),
   (2-G, 1-M, 1-G, 1-G, 2-G),
   (2-G, 1-M, 1-G, 2-G, 2-G),
   (2-G, 1-M, 2-G, 2-G, 2-G)})]

In [ ]:
def get_pairs(hand):
    value_counter = Counter(hand)
    for value, count in value_counter.items():
    # pairs
    # pairCards = [card for card in hand if card.get_rank_value() == value]
    relevent_cards = [card for card in hand if card.get_rank_value() == value]
    if count >= 2 and (last_hand is None or (HandType.PAIR == last_hand.hand_type)):
        
        pairs = set(combinations(relevent_cards, 2))
        print("pair:",pairs)
        possible_combinations += (("Pair", pairs))

['Pair',
 {(4-Y, 4-R)},
 'Pair',
 {(6-Y, 6-G), (6-Y, 6-Y), (6-R, 6-G), (6-R, 6-Y)},
 ('ThreeOfAKind', {(6-Y, 6-Y, 6-G), (6-R, 6-Y, 6-G), (6-R, 6-Y, 6-Y)}),
 ('GangOfX', {(6-R, 6-Y, 6-Y, 6-G)}),
 'Pair',
 {(9-R, 9-Y)},
 'Pair',
 {(5-Y, 5-G)},
 'Pair',
 {(2-R, 2-Y)}]

In [70]:
list_test = set(pairs)

TypeError: unhashable type: 'Card'

In [ ]:
pairCards = [card for card in card_list if card.get_rank_value() == 8]

In [62]:
list([("Pair", pair) for pair in zip(pairCards[::2], pairCards[1::2])])

[('Pair', (9-R, 9-R)), ('Pair', (9-Y, 9-G))]

In [ ]:
possible_combinations.extend([("Pair", pair) for pair in zip(pairs[::2], pairs[1::2])])

True

In [46]:
sorted(c.values()) == [2, 3]

True

In [27]:
d = {0: i for i in range(2)}

In [30]:
len(d)

1

In [29]:
list(d.values())

[1]

In [2]:
try:
    raise ValueError("test")
except Exception as e:
    print(e.)

<class 'ValueError'>


In [2]:
from deck import Card, Suits

In [11]:
test = [Card("1", Suits.Green), Card("1", Suits.Multicolor)]
list_b = [Card("1", Suits.Red), Card("1", Suits.Multicolor)] 

In [14]:
zip(*map(reversed, (list1, list2))):

TypeError: reversed expected 1 argument, got 2

In [17]:
for i in map(reversed,(test, list_b)):
    print(i)

In [21]:
for a, b in zip(*map(reversed,(test, list_b))):
    print(a)
    print(b)
    print(a == b)
    print(a < b)


TypeError: reversed expected 1 argument, got 2

In [4]:
test == list_b

True

In [7]:
type(Card("1", Suits.Yellow)) == Card

True

In [31]:
test_dict = {'a':4, 'b':2, 'c':3, "d":2}

In [32]:
min(test_dict, key=test_dict.get)

'b'

In [34]:
for t in test_dict:
    print(t)

a
b
c
d


In [72]:
from operator import itemgetter


In [73]:
from player import Player

In [76]:
last_round_winner = 'def'
players = [Player('abc'), Player('def'), Player('hij'), Player('klm')]

In [77]:
round_looser_list = ['abc', 'hij']

In [81]:
# winner_idx = players.index(Player(last_round_winner))
# round_looser_distance_to_winner = [(players.index(Player(player_id))-winner_idx) % 4 for player_id in round_looser_list]
# print(round_looser_distance_to_winner)
round_looser_list = min(enumerate([0,1]), key= itemgetter(1))[0]

In [79]:
list(min(enumerate(round_looser_distance_to_winner), key= itemgetter(1))[0])

1